In [30]:
import pandas as pd
import numpy as np

In [6]:
df_movies = pd.read_csv('data/movies_metadata.csv')
df_ratings = pd.read_csv('data/ratings.csv')

/var/folders/7t/zdlsv2353t9ftz425gflmdg80000gn/T/ipykernel_37323/3554825638.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('data/movies_metadata.csv')


In [8]:
df_movies_short = df_movies[['id', 'title', 'overview', 'genres']]
df_ratings_short = df_ratings[['userId', 'movieId', 'rating']]


In [15]:
df_ratings_short['movieId'] = pd.to_numeric(df_ratings_short['movieId'], errors='coerce')
df_movies_short['id'] = pd.to_numeric(df_movies_short['id'], errors='coerce')

df = pd.merge(df_ratings_short, df_movies_short, left_on='movieId', right_on='id', how='inner')


/var/folders/7t/zdlsv2353t9ftz425gflmdg80000gn/T/ipykernel_37323/853252443.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ratings_short['movieId'] = pd.to_numeric(df_ratings_short['movieId'], errors='coerce')
/var/folders/7t/zdlsv2353t9ftz425gflmdg80000gn/T/ipykernel_37323/853252443.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies_short['id'] = pd.to_numeric(df_movies_short['id'], errors='coerce')


In [ ]:
import json, ast

# --- 1) Parse genres safely to list[str] ---
def parse_genres(x):
    if pd.isna(x): return []
    if isinstance(x, list): return [d.get('name') for d in x if isinstance(d, dict) and 'name' in d]
    if isinstance(x, str):
        try:
            y = json.loads(x)
        except Exception:
            try:
                y = ast.literal_eval(x)
            except Exception:
                return []
        if isinstance(y, list):
            return [d.get('name') for d in y if isinstance(d, dict) and 'name' in d]
    return []

df['genre_list'] = df['genres'].apply(parse_genres)

In [31]:
from statistics import mean

# ============================================================
# 1) COUNT PER-USER GENRES + SHARES
#    Assumes df has columns: ['userId', 'genre_list', ...]
# ============================================================

# Explode genre list to long form and count
user_genre_counts = (
    df[['userId', 'genre_list']]
      .explode('genre_list')
      .dropna()
      .assign(cnt=1)
      .groupby(['userId', 'genre_list'], observed=True)['cnt']
      .sum()
      .rename('count')
      .reset_index()
)

# Compute per-user totals and shares
totals = (
    user_genre_counts
    .groupby('userId', observed=True)['count']
    .sum()
    .rename('total')
    .reset_index()
)

user_genre_counts = user_genre_counts.merge(totals, on='userId')
user_genre_counts['share'] = user_genre_counts['count'] / user_genre_counts['total']

# ============================================================
# 2) TAKE TOP-N GENRES PER USER (ranked by share)
# ============================================================

N_TOP = 3

topN = (
    user_genre_counts
      .sort_values(['userId', 'share'], ascending=[True, False])
      .groupby('userId', observed=True)
      .head(N_TOP)
)

# Pack the top genres in rank order into a list per user
df_user_top = (
    topN
      .sort_values(['userId', 'share'], ascending=[True, False])
      .groupby('userId', observed=True)['genre_list']
      .apply(list)
      .reset_index(name='top_genres')
)

# ============================================================
# 3) TAG MAP (age as ranges) + PERSONA BUILDER
# ============================================================

tag_map = {
    "History":          {"adj": ["reflective", "context-seeking"],           "hobbies": ["museums", "historical podcasts", "biographies"],        "age": (30, 60)},
    "Thriller":         {"adj": ["edge-chasing", "tension-loving"],          "hobbies": ["true-crime reads", "escape rooms", "puzzle games"],     "age": (20, 45)},
    "TV Movie":         {"adj": ["episodic", "snackable-story fan"],         "hobbies": ["mini-series binges", "weekly watchlists"],              "age": (15, 35)},
    "Foreign":          {"adj": ["worldly", "open-minded"],                  "hobbies": ["language apps", "festival films", "travel vlogs"],      "age": (20, 50)},
    "Romance":          {"adj": ["heart-forward", "sentimental"],            "hobbies": ["book clubs", "journaling", "cozy cafés"],               "age": (15, 40)},
    "Western":          {"adj": ["independent", "frontier spirit"],          "hobbies": ["trail walks", "classic cinema", "photography"],         "age": (30, 65)},
    "Action":           {"adj": ["thrill-seeking", "decisive"],              "hobbies": ["HIIT workouts", "stunt reels", "arcade shooters"],      "age": (12, 30)},
    "Family":           {"adj": ["warm", "togetherness-oriented"],           "hobbies": ["board games", "picnics", "family nights"],              "age": (25, 50)},
    "Mystery":          {"adj": ["puzzle-minded", "inquisitive"],            "hobbies": ["logic puzzles", "mystery novels", "geocaching"],        "age": (18, 50)},
    "Horror":           {"adj": ["fearless", "adrenaline-immune"],           "hobbies": ["haunted walks", "practical FX vids", "indie horrors"],  "age": (15, 35)},
    "Animation":        {"adj": ["imaginative", "whimsical"],                "hobbies": ["storyboarding", "sketching", "anime nights"],           "age": (10, 25)},
    "Comedy":           {"adj": ["lighthearted", "wit-driven"],              "hobbies": ["open mics", "sitcom rewatches", "improv clips"],        "age": (12, 35)},
    "War":              {"adj": ["stoic", "stakes-aware"],                   "hobbies": ["military history", "strategy games", "documentaries"],  "age": (25, 60)},
    "Documentary":      {"adj": ["truth-seeking", "curious"],                "hobbies": ["nonfiction reads", "explainers", "deep-dives"],         "age": (20, 55)},
    "Fantasy":          {"adj": ["mythic", "world-builder"],                 "hobbies": ["tabletop RPGs", "lore wikis", "fan art"],               "age": (12, 35)},
    "Crime":            {"adj": ["sleuthy", "grit-tolerant"],                "hobbies": ["noir novels", "case studies", "detective games"],       "age": (18, 50)},
    "Music":            {"adj": ["rhythmic", "aesthetic-tuned"],             "hobbies": ["playlist curating", "gigs", "instrument practice"],     "age": (10, 35)},
    "Science Fiction":  {"adj": ["futurist", "what-if thinker"],             "hobbies": ["tech blogs", "space docs", "indie sci-fi"],             "age": (15, 45)},
    "Drama":            {"adj": ["empathetic", "character-focused"],         "hobbies": ["arthouse nights", "screenwriting pods", "journaling"],  "age": (18, 55)},
    "Adventure":        {"adj": ["explorer", "curiosity-led"],               "hobbies": ["day hikes", "travel planning", "outdoor films"],        "age": (12, 40)},
}


def _age_phrase(age_num: float) -> str:
    decade = int(age_num) // 10 * 10
    within = int(age_num) % 10
    if within <= 3:
        band = "early"
    elif within <= 6:
        band = "mid"
    else:
        band = "late"
    return f"{band} {decade}s"

def _dedup_keep_order(seq):
    seen = set()
    out = []
    for x in seq:
        if x not in seen:
            out.append(x)
            seen.add(x)
    return out

def build_persona(
    top_genres,
    max_genres=3,
    max_traits=3,
    max_hobbies=3,
    unknown_label="Eclectic",
    # jitter controls
    jitter_low=-2,
    jitter_high=2,
    rng=None
):
    """
    top_genres: ordered list of genre strings (highest to lowest preference).
    rng: np.random.RandomState for reproducible jitter (optional).
    """
    if not top_genres:
        return f"{unknown_label} fan — curious about everything; age vibe: mid 30s. Hobbies: reading widely."

    # Keep only known genres, preserve order, cap
    gsel, seen = [], set()
    for g in top_genres:
        if g in tag_map and g not in seen:
            gsel.append(g)
            seen.add(g)
        if len(gsel) >= max_genres:
            break

    if not gsel:
        return f"{unknown_label} fan — genre-agnostic; age vibe: mid 30s. Hobbies: exploring new releases."

    traits, hobbies, ages = [], [], []
    for g in gsel:
        meta = tag_map[g]
        traits.extend(meta["adj"])
        hobbies.extend(meta["hobbies"])
        lo, hi = meta["age"]
        if rng is None:
            ages.append(np.random.randint(lo, hi + 1))
        else:
            ages.append(rng.randint(lo, hi + 1))

    traits  = _dedup_keep_order(traits)[:max_traits]
    hobbies = _dedup_keep_order(hobbies)[:max_hobbies]

    # Add light jitter to the averaged age for natural variation
    base_age = mean(ages)
    jitter = (rng.randint(jitter_low, jitter_high + 1) if rng is not None
              else np.random.randint(jitter_low, jitter_high + 1))
    age_vibe = _age_phrase(base_age + jitter)

    head = " / ".join(gsel[:2]) if len(gsel) >= 2 else gsel[0]
    trait_str = ", ".join(traits) if traits else "versatile"
    hobby_str = ", ".join(hobbies) if hobbies else "discovering new films"

    return f"{head} fan — {trait_str}; age vibe: {age_vibe}. Hobbies: {hobby_str}."

# ============================================================
# 4) BUILD PERSONAS PER USER (REPRODUCIBLE JITTER)
#    Use a seeded RNG per userId so personas are stable run-to-run
# ============================================================

def _rng_for_user(user_id: int) -> np.random.RandomState:
    # stable seed per user; works for int-like IDs
    return np.random.RandomState(seed=np.int64(user_id) % (2**32 - 1))

def build_persona_for_row(row):
    rng = _rng_for_user(row['userId'])
    return build_persona(row['top_genres'], rng=rng)

user_personality = (
    df_user_top
      .assign(personality=lambda x: x.apply(build_persona_for_row, axis=1))
)

# Result columns: ['userId', 'top_genres', 'personality']
# Example row:
# userId = 123
# top_genres = ['Science Fiction', 'Action', 'Comedy']
# personality = "Science Fiction / Action fan — futurist, what-if thinker, thrill-seeking; age vibe: mid 20s. Hobbies: tech blogs, space docs, indie sci-fi."


In [33]:
df

,userId,movieId,rating,id,title,overview,genres,genre_list
0,1,110,1.0,110.0,Three Colors: Red,Red This is the third film from the trilogy by...,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...","[Drama, Mystery, Romance]"
1,1,147,4.5,147.0,The 400 Blows,"For young Parisian boy Antoine Doinel, life is...","[{'id': 18, 'name': 'Drama'}]",[Drama]
2,1,858,5.0,858.0,Sleepless in Seattle,A young boy who tries to set his dad up on a d...,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[Comedy, Drama, Romance]"
3,1,1246,5.0,1246.0,Rocky Balboa,When he loses a highly publicized virtual boxi...,"[{'id': 18, 'name': 'Drama'}]",[Drama]
4,1,1968,4.0,1968.0,Fools Rush In,Alex Whitman (Matthew Perry) is a designer fro...,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...","[Drama, Comedy, Romance]"
...,...,...,...,...,...,...,...,...
11437632,270896,48780,5.0,48780.0,Boat,A journey into night.,"[{'id': 10769, 'name': 'Foreign'}]",[Foreign]
11437633,270896,49530,4.0,49530.0,In Time,In the not-too-distant future the aging gene h...,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...","[Action, Thriller, Science Fiction]"
11437634,270896,54001,4.0,54001.0,The Traveler,The Traveler is a 1974 Iranian drama film dire...,"[{'id': 18, 'name': 'Drama'}]",[Drama]
11437635,270896,54503,4.0,54503.0,The Mystery of Chess Boxing,Lee Yi Min stars as an eager young kung fu stu...,"[{'id': 28, 'name': 'Action'}, {'id': 10769, '...","[Action, Foreign]"


In [32]:
user_personality

,userId,top_genres,personality
0,1,"[Drama, Comedy, Romance]","Drama / Comedy fan — empathetic, character-foc..."
1,2,"[Drama, Comedy, Action]","Drama / Comedy fan — empathetic, character-foc..."
2,3,"[Comedy, Drama, Crime]","Comedy / Drama fan — lighthearted, wit-driven,..."
3,4,"[Drama, Comedy, Romance]","Drama / Comedy fan — empathetic, character-foc..."
4,5,"[Drama, Thriller, Action]","Drama / Thriller fan — empathetic, character-f..."
...,...,...,...
265843,270892,"[Drama, Comedy, Action]","Drama / Comedy fan — empathetic, character-foc..."
265844,270893,"[Drama, Thriller, Comedy]","Drama / Thriller fan — empathetic, character-f..."
265845,270894,"[Drama, Comedy, Romance]","Drama / Comedy fan — empathetic, character-foc..."
265846,270895,"[Drama, Comedy, Thriller]","Drama / Comedy fan — empathetic, character-foc..."


In [42]:
df_final = pd.merge(
    df,
    user_personality[['userId', 'top_genres', 'personality']],
    on='userId',
    how='left'
)

df_final.to_csv('data/movie_user_data.csv', index=False)

In [43]:
import pickle

# Save as pickle
with open('data/movie_user_data.pkl', 'wb') as f:
    pickle.dump(df_final, f)

print("✅ Saved as pickle without dtype drama.")


✅ Saved as pickle without dtype drama.
